
    =================================================
    
   

    Nama  : Mardhya Malik Nurbani


    Program ini dibuat untuk membangun konteks data (data context) menggunakan Great Expectations, sebuah perangkat lunak open-source yang 
    digunakan untuk mengelola, memantau, dan menguji kualitas data.
    
    =================================================

## Data Context

In [7]:
# Create a data context

from great_expectations.data_context import FileDataContext
import pandas as pd
from great_expectations.dataset import PandasDataset
import os


context = FileDataContext.create(project_root_dir='C:\\Users\\Mardhya\\Hacktiv8\\Milestone 3\\p2-ftds026-rmt-m3-mnurbani97\\MachineLearning\\dags')

**Insight** : membuat objek konteks data dari proyek Great Expectations yang terletak di `'C:\\Users\\Mardhya\\Hacktiv8\\Milestone 3\\p2-ftds026-rmt-m3-mnurbani97\\MachineLearning\\dags'`
 yang digunakan untuk melakukan berbagai operasi validasi dan pemantauan data

## Connect to A `Datasource`

In [8]:

# Local CSV file path
local_csv_path = 'C:\\Users\\Mardhya\\Hacktiv8\\Milestone 3\\p2-ftds026-rmt-m3-mnurbani97\\MachineLearning\\dags\\P2M3_mardhya_data_clean.csv'

# Extracting the file name without extension to use as asset_name
asset_name = os.path.splitext(os.path.basename(local_csv_path))[0]

# Give a unique name to the datasource
datasource_name = 'csv-data-superstore'

# Adding the datasource
datasource = context.sources.add_pandas(datasource_name)

# Adding the CSV asset to the datasource
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=local_csv_path)

# Building batch request
batch_request = asset.build_batch_request()


**Insight** : file csv yang berada di `C:\\Users\\Mardhya\\Hacktiv8\\Milestone 3\\p2-ftds026-rmt-m3-mnurbani97\\MachineLearning\\dags\\` bernama `P2M3_mardhya_data_clean.csv` akan di load untuk dimonitoring . lalu akan di ambil file csv tanpa ekstensinya dengan cara `os.path.basename` dan dipisahkan ekstensi file menggunakan `os.path.splitext`, kemudian disimpan di dalam variable `asset_name`. lalu data CSV yang akan ditambahkan ke konteks Great Expectations dengan nama `csv-data-superstore` dan disimpan pada variable `datasource_name`. `Asset` ini adalah representasi dari file CSV yang akan dimonitor. lalu pada `batch_request` digunakan untuk memeriksa data dalam file csv untuk melakukan pemeriksaaan dan validasi pada data.

## Create an Expectation Suite

In [80]:
# Creat an expectation suite
expectation_suite_name = 'expectation-superstore-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)
df = pd.read_csv(local_csv_path)
df['no_index'] = df.index
# Check the validator
validator.head()



Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,sales,quantity,discount,profit
0,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,731.9400,3,0.00,219.5820
2,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,957.5775,5,0.45,-383.0310
4,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,22.3680,2,0.20,2.5164


**Insight** : membuat, mengatur, dan memeriksa validator menggunakan Great Expectations, dengan cara menentukan nama untuk suite ekspektasi (expectation suite) yang akan dibuat, lalu menambahkan atau memperbarui suite ekspektasi dengan nama yang telah ditentukan. selanjutnya pada `validator` membuat objek validator dengan menggunakan permintaan batch (batch request) yang telah dibangun sebelumnya dan suite ekspektasi yang telah dibuat. setelah itu fungsi akan dipanggil pada `validator.head()` untuk memeriksa beberapa baris pertama dari data yang telah dimuat ke dalam validator
dengan melakukan langkah semua ini dapat memastikan bahwa data memenuhi aturan atau ekspektasi tertentu sebelum dilanjutkan ke tahap analisis atau penggunaan selanjutnya. 

## Expectations

#### To be unique

In [85]:
batch_data = PandasDataset(df)

# Membuat expectation bahwa nilai dalam kolom 'no_index'
batch_data.expect_column_values_to_be_unique('no_index') #membuat column baru untuk melakukan unique values
validation_results = batch_data.validate()

validation_results

{
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "expectation_type": "expect_column_values_to_be_unique",
        "kwargs": {
          "column": "no_index",
          "result_format": "BASIC"
        },
        "meta": {}
      },
      "result": {
        "element_count": 9977,
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "unexpected_percent_total": 0.0,
        "unexpected_percent_nonmissing": 0.0,
        "partial_unexpected_list": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_message": null,
        "exception_traceback": null
      }
    }
  ],
  "evaluation_parameters": {},
  "statistics": {
    "evaluated_expectations": 1,
    "successful_expectations": 1,
    "unsuccessful_expectations": 0,
    "success_percent": 100.0
  },
  "meta": {
    "great_expectations_version": "0.1

**Insight** : Melakukan expect_column_values_to_be_unique pada kolom yang barusaja ditambahkan `no_index`. berikut hasilnya :
-  Hasilnya menunjukkan bahwa semua nilai element_count": 9977 dalam kolom tersebut unik, Jumlah nilai yang hilang dalam kolom 0 dan umlah nilai yang tidak terduga (non-unik) yang ditemukan dalam kolom 0. validasi berhasil dan kolom tersebut memenuhi harapan bahwa nilainya harus unik `"success": true,`

#### To be set

In [86]:
unique_subcategories = ['Bookcases', 'Chairs', 'Labels', 'Tables', 'Storage', 'Furnishings', 'Art', 'Phones', 'Binders', 'Appliances', 'Paper', 'Accessories', 'Envelopes', 'Fasteners', 'Supplies', 'Machines', 'Copiers']

validator.expect_column_values_to_be_in_set('sub_category', unique_subcategories)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9977,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Insight** : Melakukan `validator.expect_column_values_to_be_in_set` pada column  `sub_category`. yang hasilnya :

- Hasil pemeriksaan pada column `sub_category`, bahwa setiap nilai dalam kolom `'sub_category'` sebenarnya termasuk dalam daftar nilai yang telah ditentukan sebelumnya. Hasilnya menunjukkan bahwa semua nilai dalam kolom tersebut `("element_count": 9977)` memang ada dalam daftar yang diharapkan, dan tidak ada nilai yang tidak seharusnya atau bahkan yang hilang sama sekali. Dengan kata lain, data dalam kolom `'sub_category'` memenuhi apa yang diharapkan dari awal dengan status `"success": true`,


#### To be between min_value and max_value

In [87]:
validator.expect_column_mean_to_be_between('discount', 0, 0.5)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 0.1562784404129498
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Insight** : Melakukan validator.expect_column_mean_to_be_between pada column `discount` dengan range 0 - 0.5. Berikut hasilnya:
-  Hasil pemeriksaan menunjukkan bahwa rata-rata kolom `'discount'` adalah 0.156, yang artinya nilai rata-rata yang berada pada column `discount` berada pada range `0 - 0.5`, dan pemeriksaan berlangsung dengan sukses. `"success": true,`

#### To be List

In [88]:
validator.expect_column_values_to_be_in_type_list('profit',['integer', 'float'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Insight** : melakukan validator.expect_column_values_to_be_in_type_list pada column `profit` dengan type data yang diharapkan yaitu `'integer', 'float'`, berikut hasilnya :
- Hasil pemeriksaan menunjukkan bahwa semua nilai dalam kolom 'profit' sesuai dengan tipe data yang diharapkan, yaitu integer atau float `"success": true,`. nilai pada column tersebut merupakan tipe data float, pemeriksaan berlangsung dengan sukses.

#### most_common_value_to_be_in_set

In [89]:
value_set_for_ship_mode = ['Standard Class', 'Second Class', 'First Class', 'Same Day']
validator.expect_column_most_common_value_to_be_in_set('ship_mode', value_set_for_ship_mode)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "Standard Class"
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


**Insight** : Melakukan validator.expect_column_most_common_value_to_be_in_set pada column `ship_mode`, dengan mencari mayoritas pada column tersebut. berikut hasilnya :

- Validasi sukses dan menunjukkan bahwa bahwa nilai yang paling umum (most common value) dalam kolom `'ship_mode'` adalah `"Standard Class"`, yang memang termasuk dalam himpunan nilai yang diharapkan: 'Standard Class', 'Second Class', 'First Class', dan 'Same Day'. pemeriksaan berlangsung dengan sukses `"success": true`, 

#### to_be_of_type

In [90]:
validator.expect_column_values_to_be_of_type('sub_category', 'object')


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "object_"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Insight** Melakukan validator.expect_column_values_to_be_of_type pada column `sub_category`
- Hasil pemeriksaan menunjukkan bahwa semua nilai dalam kolom `sub_category` tersebut adalah tipe data 'object'

In [91]:
validator.expect_column_proportion_of_unique_values_to_be_between('sub_category', min_value=0, max_value=0.1)


Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 0.0017039190137315826
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Insight** : Melakukan validator.expect_column_proportion_of_unique_values_to_be_between pada `sub_category`, dengan range 0 - 0.1. berikut hasilnya :
- Hasil pemeriksaan menunjukkan bahwa proporsi nilai unik dalam kolom `sub_category` adalah sekitar `0.00170`. yang artinya hanya sebagian kecil dari nilai dalam kolom tersebut yang unik dan pemeriksaan berlangsung dengan sukses. `"success": true,`

#### median_to_be_between

In [92]:
validator.expect_column_median_to_be_between('discount',min_value=0, max_value=1 )

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 0.2
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Insight** : Melakukan validator.expect_column_median_to_be_between pada column `discount`. Berikut hasilnya :
- Hasil pemeriksaan menunjukkan bahwa nilai median dalam kolom `discount` adalah `0.2`, yang berada dalam rentang yang diharapkan antara 0 dan 1.   pemeriksaan berlasung dengan sukses `"success": true,`

#### row_count_to_be_between

In [93]:
validator.expect_table_row_count_to_be_between(0, 10000)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 9977
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**Insight** : Melakukan validator.expect_table_row_count_to_be_between pada dataset `P2M3_mardhya_data_clean.csv`. Berikut hasilnya :
- Hasil pemeriksaan menunjukkan bahwa jumlah baris dalam tabel adalah `9977`, yang berada dalam rentang yang diharapkan antara 0 dan 10000. yang artinya bahwa pemeriksaan berlangsung dengan sukses. `"success": true`

#### column_values_to_be_null

In [94]:
validator.expect_column_values_to_not_be_null('sub_category')

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9977,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**insight** : Melakukan validator.expect_column_values_to_not_be_null pada column `sub_category`. memverifikasi bahwa setiap entri dalam kolom `sub_category` memiliki nilai yang tidak null. berikut hasilnya :
- pemeriksaan data pada kolom 'sub_category' berhasil dilakukan tanpa masalah. Tidak ada nilai yang tidak diharapkan (NULL) dalam kolom tersebut. Jumlah total baris yang diperiksa adalah 9977, dengan persentase nilai NULL sebesar 0.0%, `"success": true,`

In [95]:
validator.save_expectation_suite(discard_failed_expectations=False)

**Insight** : menyimpan suite ekspektasi yang telah didefinisikan. dengan ini bisa melacak dan memeriksa ulang aturan yang gagal untuk memahami perubahan / masalah pada data.

## Checkpoint

In [96]:
# Create a checkpoint

checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

**Insight** : Great Expectations. Ini memungkinkan Anda untuk membuat checkpoint untuk memantau kemajuan dalam validasi dan memperbarui checkpoint yang ada jika diperlukan. Ini memungkinkan checkpoint untuk terkait langsung dengan aturan validasi yang telah Anda tetapkan sebelumnya.

In [97]:
# Run a checkpoint

checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/18 [00:00<?, ?it/s]

**Insight** : menjalankan checkpoint yang telah Anda buat sebelumnya.

## Data Docs

In [98]:
# Build data docs

context.build_data_docs()

{'local_site': 'file://C:\\Users\\Mardhya\\Hacktiv8\\Milestone 3\\p2-ftds026-rmt-m3-mnurbani97\\MachineLearning\\dags\\gx\\uncommitted/data_docs/local_site/index.html'}

**Insight** : membangun dokumentasi data (data docs) menggunakan konteks Great Expectations. `{'local_site': 'file://C:\\Users\\Mardhya\\Hacktiv8\\Milestone 3\\p2-ftds026-rmt-m3-mnurbani97\\MachineLearning\\dags\\gx\\uncommitted/data_docs/local_site/index.html'}` merupakan lokasi lokal dari dokumentasi data (data docs) yang telah dibangun oleh Great Expectations.